In [13]:
import datetime

In [14]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.linear_model import Ridge

In [15]:
symbol = input("Enter the symbol you want to perform DDM on: ").upper()

In [16]:
required_return = float(input("Enter the required rate of return as a percentage for the year: "))
required_return /= 100

In [17]:
ticker = yf.Ticker(symbol)

In [18]:
dividends = pd.DataFrame(ticker.dividends)
dividends = dividends.groupby(dividends.index.year)['Dividends'].sum()
dividends = pd.DataFrame(dividends).sort_index(ascending=False)

In [19]:
growth_rate = dividends['Dividends'].pct_change().mean()

In [20]:
dividends.drop(dividends.index[dividends.index == datetime.datetime.now().year], inplace = True)
dividends = dividends.sort_index()
dividends.index = pd.to_datetime(dividends.index, format='%Y')

In [21]:
for i in range(5):
    next_year = dividends.iloc[-1].values + (growth_rate * dividends.iloc[-1].values)
    dividends.loc[pd.to_datetime(dividends.index.max().year + 1, format='%Y')] = next_year

In [22]:
dividends.index = dividends.index.year
dividends.sort_index(ascending=False, inplace=True)

In [23]:
present_price = ticker.history(period='1d')['Close'].values
dividends_5y = dividends.head()

In [24]:
counter = 1
DDM_price = 0
for i in range(len(dividends_5y) - 1):
    DDM_price += (dividends_5y.iloc[-1-i].values) / ((1 + required_return) ** counter)
    counter += 1


DDM_price += (present_price + dividends_5y.iloc[0].values) / ((1 + required_return) ** counter)

In [25]:
if present_price > DDM_price:
    print('Overpriced')
else:
    print('Underpriced')

Overpriced
